In [ ]:
import torch
import cv2
import numpy as np

onnx_path = "../../model/face_detection_yunet_120x160.onnx"  # 文件的路径
img = cv2.imread("./world_cup.jpg")

# load onnx model
import onnx
import onnxruntime as ort

onnx_model = onnx.load(onnx_path)

# 打印输入和输出
print("Inputs: ", onnx_model.graph.input)
print("Outputs: ", onnx_model.graph.output)

onnx.checker.check_model(onnx_model)
# 打印模型的层
# print(onnx.helper.printable_graph(onnx_model.graph))

# 创建一个运行时会话
sess = ort.InferenceSession(onnx_path)

# 输入是一个形状为 (1, 3, 120, 160) 的张量
# 将图像缩放到 640x640
img = cv2.resize(img, (120, 160))
# 将 BGR 图像转换为 RGB 图像
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
# 将图像数据类型转换为 float32，并将值范围从 [0, 255] 缩放到 [0.0, 1.0]
img = img.astype('float32') / 255.0
# 将 HWC 图像转换为 CHW 图像
img = img.transpose((2, 0, 1))
# 添加一个新的维度，将 CHW 图像转换为 NCHW 图像
img_tensor = torch.from_numpy(img)

img_array = np.expand_dims(img_tensor, axis=0)
input_name = sess.get_inputs()[0].name

# 运行模型
result = sess.run(None, {input_name: img_array})

# 打印结果
print(result)
